In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/Training/Train_SUV.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Truck.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Van.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Special.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Sedan.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Hatchback.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Freight.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Coupe.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Convertable.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Bus.zip" -d "/content/train"
!unzip -qq "/content/drive/MyDrive/Training/Train_Bicycle.zip" -d "/content/train"

!unzip -qq "/content/drive/MyDrive/Validation/Test_Van.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_SUV.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Truck.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Sedan.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Hatchback.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Freight.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Bus.zip" -d "/content/test"
!unzip -qq "/content/drive/MyDrive/Validation/Test_Bicycle.zip" -d "/content/test"


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
import shutil

def classify_images(directory):
    # 이미지 파일 확장자 목록 (필요에 따라 확장자 추가 가능)
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']

    # os.walk를 사용하여 모든 하위 디렉토리를 재귀적으로 탐색
    for root, _, files in os.walk(directory):
      # print(files)
      # print(root)
      for filename in files:
          if any(filename.lower().endswith(ext) for ext in image_extensions):

              parts = filename.split('-')
              if len(parts) >= 2:
                  category = parts[0]

                  parent_folder_name = os.path.basename(root)
                  new_folder_name = f"{parent_folder_name}_{category}"
                  new_folder_path = os.path.join(directory, new_folder_name)

                  if not os.path.exists(new_folder_path):
                      os.makedirs(new_folder_path)

                  src_file_path = os.path.join(root, filename)
                  dest_file_path = os.path.join(new_folder_path, filename)
                  shutil.move(src_file_path, dest_file_path)

def remove_empty_folders(path):
    if not os.path.isdir(path):
        return

    # 하위 디렉토리 먼저 삭제
    for dirpath, dirnames, filenames in os.walk(path, topdown=False):
        for dirname in dirnames:
            dir_full_path = os.path.join(dirpath, dirname)
            try:
                os.rmdir(dir_full_path)
            except OSError:
                # 폴더가 비어 있지 않으면 예외 발생
                pass

    # 최상위 디렉토리 삭제 시도
    try:
        os.rmdir(path)
    except OSError:
        # 폴더가 비어 있지 않으면 예외 발생
        pass

def create_missing_folders(source_dir, target_dir):
    # /content/train의 모든 폴더 이름을 읽기
    for folder_name in os.listdir(source_dir):
        source_folder_path = os.path.join(source_dir, folder_name)
        if os.path.isdir(source_folder_path):
            # /content/test에 해당 폴더가 없으면 생성
            target_folder_path = os.path.join(target_dir, folder_name)
            if not os.path.exists(target_folder_path):
                os.makedirs(target_folder_path)

train_directory_path = '/content/train'
test_directory_path = '/content/test'

classify_images(train_directory_path)
classify_images(test_directory_path)

remove_empty_folders(train_directory_path)
remove_empty_folders(test_directory_path)

create_missing_folders(train_directory_path, test_directory_path)

초기학습용 코드

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import numpy as np
from torchvision.models import efficientnet_b0  # Importing EfficientNet from torchvision.models
from torch.utils.data import DataLoader
import os

# Function to compile and train the model
def compile(model, train_loader, test_loader, epochs, optimizer, loss, device):
    train_acc = []
    train_loss = []
    test_acc = []
    test_loss = []
    best_model_save_path = "/content/drive/MyDrive/Car_recognition_Data/model_Brand_efficientnet_b0.pth"
    best_test_acc = 0.0

    for epoch in range(epochs):
        model.train()
        num_train = 0
        num_correct_train = 0
        total_train_loss = 0.0
        cnt = 0

        print(f"Epoch {epoch+1}/{epochs}")

        for (xList, yList) in train_loader:
            xList, yList = xList.to(device), yList.to(device)
            optimizer.zero_grad()

            outputs = model(xList)
            train_loss_func = loss(outputs, yList)
            train_loss_func.backward()
            optimizer.step()

            num_train += len(yList)
            total_train_loss += train_loss_func.item() * len(yList)
            predicts = torch.max(outputs.data, 1)[1]
            num_correct_train += (predicts == yList).sum().item()

        train_acc.append(num_correct_train / num_train)
        train_loss.append(total_train_loss / num_train)
        print(f"    - train_acc: {train_acc[-1]:.5f}, train_loss: {train_loss[-1]:.5f}")

        model.eval()
        num_test = 0
        num_correct_test = 0
        total_test_loss = 0.0

        with torch.no_grad():
            for (xList, yList) in test_loader:
                xList, yList = xList.to(device), yList.to(device)
                outputs = model(xList)
                test_loss_func = loss(outputs, yList)

                num_test += len(yList)
                total_test_loss += test_loss_func.item() * len(yList)
                predicts = torch.max(outputs.data, 1)[1]
                num_correct_test += (predicts == yList).sum().item()

        test_acc.append(num_correct_test / num_test)
        test_loss.append(total_test_loss / num_test)
        print(f"    - test_acc: {test_acc[-1]:.5f}, test_loss: {test_loss[-1]:.5f}")

        #테스트 결과가 기존보다 좋아지면 drive에 저장
        if test_acc[-1] > best_test_acc:
          best_test_acc = test_acc[-1]
          torch.save(model.state_dict(), best_model_save_path)
          print(f"Best model saved with test accuracy: {best_test_acc:.5f}")

    return train_loss, train_acc, test_loss, test_acc

# Function to fix random seed for reproducibility
def fix_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

fix_seed(1111)

# Configuration dictionary
config = {
    "batch_size": 128,
    "learning_rate": 0.0001,
    "epochs": 10,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
}

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalization for RGB images
])

# Load MNIST dataset
trainset = torchvision.datasets.ImageFolder(root="/content/train/", transform=transform, allow_empty=True)
testset = torchvision.datasets.ImageFolder(root="/content/test/", transform=transform, allow_empty=True)

trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
testloader = DataLoader(testset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

print(trainset.classes)
print(testset.classes)

# Load pre-trained EfficientNet model and modify the classifier
model = efficientnet_b0(pretrained=True)
num_features = model.classifier[1].in_features
#num_features = model.fc.in_features

model.classifier = nn.Sequential(
    nn.Dropout(0.5),  # Add dropout for regularization
    nn.Linear(num_features, len(trainset.classes))
)
model.to(config['device'])

# Loss function and optimizer
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

# Train the model
result = compile(model, trainloader, testloader, config["epochs"], optimizer, loss, config['device'])

# Save the trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/Car_recognition_Data/model_final_efficientnet_b0.pth")
print('Learning Finished!')

['BMW_SUV_X1', 'BMW_SUV_X3', 'BMW_SUV_X4', 'BMW_SUV_X5', 'BMW_SUV_X6', 'BMW_SUV_X7', 'BMW_세단_2시리즈그란쿠페', 'BMW_세단_3시리즈', 'BMW_세단_5시리즈', 'BMW_세단_7시리즈', 'BMW_컨버터블_6시리즈', 'BMW_쿠페_2시리즈', 'BMW_쿠페_4시리즈', 'BMW_쿠페_6시리즈', 'BMW_해치백_1시리즈', 'BMW_해치백_2시리즈투어러', '기아자동차_SUV_니로', '기아자동차_SUV_니로_EV', '기아자동차_SUV_모하비', '기아자동차_SUV_셀토스', '기아자동차_SUV_스토닉', '기아자동차_SUV_스포티지', '기아자동차_SUV_쏘렌토', '기아자동차_SUV_쏘울', '기아자동차_SUV_카렌스', '기아자동차_세단_K3', '기아자동차_세단_K5', '기아자동차_세단_K7', '기아자동차_세단_K9', '기아자동차_세단_로체', '기아자동차_세단_스팅어', '기아자동차_세단_스펙트라', '기아자동차_세단_쎄라토', '기아자동차_세단_오피러스', '기아자동차_세단_옵티마', '기아자동차_세단_포르테', '기아자동차_세단_프라이드', '기아자동차_승합_카니발', '기아자동차_쿠페_K3', '기아자동차_쿠페_포르테', '기아자동차_해치백_레이', '기아자동차_해치백_모닝', '기아자동차_해치백_프라이드', '닛산_SUV_로그', '닛산_SUV_엑스트레일', '닛산_SUV_쥬크', '닛산_SUV_큐브', '닛산_SUV_패스파인더', '닛산_세단_맥시마', '닛산_세단_알티마', '닛산_해치백_리프', '도요타_SUV_라브4', '도요타_세단_아발론', '도요타_세단_캠리', '도요타_해치백_프리우스', '도요타_해치백_프리우스C', '랜드로버_SUV_디스커버리', '랜드로버_SUV_레인지로버', '랜드로버_SUV_이보크', '렉서스_SUV_NX', '렉서스_SUV_RX', '렉서스_세단_ES', '렉서스_세단_LS', '렉서스_해치백_CT', '르노삼성_SU

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 130MB/s] 


Epoch 1/10
    - train_acc: 0.79002, train_loss: 0.88933
    - test_acc: 0.95467, test_loss: 0.18397
Best model saved with test accuracy: 0.95467
Epoch 2/10
    - train_acc: 0.92605, train_loss: 0.28292
    - test_acc: 0.96111, test_loss: 0.15945
Best model saved with test accuracy: 0.96111
Epoch 3/10
    - train_acc: 0.93877, train_loss: 0.23051
    - test_acc: 0.96361, test_loss: 0.15262
Best model saved with test accuracy: 0.96361
Epoch 4/10
    - train_acc: 0.94417, train_loss: 0.20725
    - train_acc: 0.94417, train_loss: 0.20725
    - test_acc: 0.96379, test_loss: 0.14961
Best model saved with test accuracy: 0.96379
Epoch 5/10
    - test_acc: 0.96379, test_loss: 0.14961
Best model saved with test accuracy: 0.96379
Epoch 5/10
    - train_acc: 0.94767, train_loss: 0.19298
    - train_acc: 0.94767, train_loss: 0.19298
    - test_acc: 0.96481, test_loss: 0.14766
Best model saved with test accuracy: 0.96481
Epoch 6/10
    - test_acc: 0.96481, test_loss: 0.14766
Best model saved with t

KeyboardInterrupt: 

KeyboardInterrupt: 

**재학습용 코드**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import numpy as np
from torchvision.models import efficientnet_b0  # Importing EfficientNet from torchvision.models
from torch.utils.data import DataLoader
import os

# Function to compile and train the model
def compile(model, train_loader, test_loader, epochs, optimizer, loss, device):
    train_acc = []
    train_loss = []
    test_acc = []
    test_loss = []
    best_model_save_path = "/content/drive/MyDrive/Car_recognition_Data/model_Brand_efficientnet_b0.pth"
    best_test_acc = 0.0

    for epoch in range(epochs):
        model.train()
        num_train = 0
        num_correct_train = 0
        total_train_loss = 0.0
        cnt = 0

        print(f"Epoch {epoch+1}/{epochs}")

        for (xList, yList) in train_loader:
            xList, yList = xList.to(device), yList.to(device)
            optimizer.zero_grad()

            outputs = model(xList)
            train_loss_func = loss(outputs, yList)
            train_loss_func.backward()
            optimizer.step()

            num_train += len(yList)
            total_train_loss += train_loss_func.item() * len(yList)
            predicts = torch.max(outputs.data, 1)[1]
            num_correct_train += (predicts == yList).sum().item()

        train_acc.append(num_correct_train / num_train)
        train_loss.append(total_train_loss / num_train)
        print(f"    - train_acc: {train_acc[-1]:.5f}, train_loss: {train_loss[-1]:.5f}")

        model.eval()
        num_test = 0
        num_correct_test = 0
        total_test_loss = 0.0

        with torch.no_grad():
            for (xList, yList) in test_loader:
                xList, yList = xList.to(device), yList.to(device)
                outputs = model(xList)
                test_loss_func = loss(outputs, yList)

                num_test += len(yList)
                total_test_loss += test_loss_func.item() * len(yList)
                predicts = torch.max(outputs.data, 1)[1]
                num_correct_test += (predicts == yList).sum().item()

        test_acc.append(num_correct_test / num_test)
        test_loss.append(total_test_loss / num_test)
        print(f"    - test_acc: {test_acc[-1]:.5f}, test_loss: {test_loss[-1]:.5f}")

        #테스트 결과가 기존보다 좋아지면 drive에 저장
        if test_acc[-1] > best_test_acc:
          best_test_acc = test_acc[-1]
          torch.save(model.state_dict(), best_model_save_path)
          print(f"Best model saved with test accuracy: {best_test_acc:.5f}")

    return train_loss, train_acc, test_loss, test_acc

# Function to fix random seed for reproducibility
####재학습용 코드입니다!!!!!#####
def fix_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

fix_seed(1111)

# Configuration dictionary
config = {
    "batch_size": 128,
    "learning_rate": 0.0001,
    "epochs": 10,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
}

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalization for RGB images
])

# Load MNIST dataset
trainset = torchvision.datasets.ImageFolder(root="/content/train/", transform=transform, allow_empty=True)
testset = torchvision.datasets.ImageFolder(root="/content/test/", transform=transform, allow_empty=True)

trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
testloader = DataLoader(testset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

print(trainset.classes)
print(testset.classes)

# Load pre-trained EfficientNet model and modify the classifier
model = efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
#num_features = model.fc.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),  # Add dropout for regularization
    nn.Linear(num_features, len(trainset.classes))
)
state_dict = torch.load('/content/drive/MyDrive/Car_recognition_Data/model_Brand_efficientnet_b0.pth')
model.load_state_dict(state_dict)
model.to(config['device'])

# Loss function and optimizer
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

# Train the model
result = compile(model, trainloader, testloader, config["epochs"], optimizer, loss, config['device'])

# Save the trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/Car_recognition_Data/model_final_efficientnet_b0.pth")
print('Learning Finished!')

['BMW_SUV_X1', 'BMW_SUV_X3', 'BMW_SUV_X4', 'BMW_SUV_X5', 'BMW_SUV_X6', 'BMW_SUV_X7', 'BMW_세단_2시리즈그란쿠페', 'BMW_세단_3시리즈', 'BMW_세단_5시리즈', 'BMW_세단_7시리즈', 'BMW_컨버터블_6시리즈', 'BMW_쿠페_2시리즈', 'BMW_쿠페_4시리즈', 'BMW_쿠페_6시리즈', 'BMW_해치백_1시리즈', 'BMW_해치백_2시리즈투어러', '기아자동차_SUV_니로', '기아자동차_SUV_니로_EV', '기아자동차_SUV_모하비', '기아자동차_SUV_셀토스', '기아자동차_SUV_스토닉', '기아자동차_SUV_스포티지', '기아자동차_SUV_쏘렌토', '기아자동차_SUV_쏘울', '기아자동차_SUV_카렌스', '기아자동차_세단_K3', '기아자동차_세단_K5', '기아자동차_세단_K7', '기아자동차_세단_K9', '기아자동차_세단_로체', '기아자동차_세단_스팅어', '기아자동차_세단_스펙트라', '기아자동차_세단_쎄라토', '기아자동차_세단_오피러스', '기아자동차_세단_옵티마', '기아자동차_세단_포르테', '기아자동차_세단_프라이드', '기아자동차_승합_카니발', '기아자동차_쿠페_K3', '기아자동차_쿠페_포르테', '기아자동차_해치백_레이', '기아자동차_해치백_모닝', '기아자동차_해치백_프라이드', '닛산_SUV_로그', '닛산_SUV_엑스트레일', '닛산_SUV_쥬크', '닛산_SUV_큐브', '닛산_SUV_패스파인더', '닛산_세단_맥시마', '닛산_세단_알티마', '닛산_해치백_리프', '도요타_SUV_라브4', '도요타_세단_아발론', '도요타_세단_캠리', '도요타_해치백_프리우스', '도요타_해치백_프리우스C', '랜드로버_SUV_디스커버리', '랜드로버_SUV_레인지로버', '랜드로버_SUV_이보크', '렉서스_SUV_NX', '렉서스_SUV_RX', '렉서스_세단_ES', '렉서스_세단_LS', '렉서스_해치백_CT', '르노삼성_SU

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10
    - train_acc: 0.95456, train_loss: 0.15914
    - test_acc: 0.96505, test_loss: 0.14891
Best model saved with test accuracy: 0.96505
Epoch 2/10
    - train_acc: 0.95697, train_loss: 0.14539
    - test_acc: 0.96546, test_loss: 0.14967
Best model saved with test accuracy: 0.96546
Epoch 3/10
    - train_acc: 0.95930, train_loss: 0.13461
    - test_acc: 0.96486, test_loss: 0.15477
Epoch 4/10


KeyboardInterrupt: 

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import efficientnet_b0

# Configuration dictionary
config = {
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "model_path": "/content/model_mnist_efficientnet_b0.pth",  # Path to the saved model
}

# Define the classes (Example: for your specific dataset)
classes = ['BMW_SUV_X1', 'BMW_SUV_X3', 'BMW_SUV_X4', 'BMW_SUV_X5', 'BMW_SUV_X6', 'BMW_SUV_X7', 'BMW_세단_2시리즈그란쿠페', 'BMW_세단_3시리즈', 'BMW_세단_5시리즈', 'BMW_세단_7시리즈', 'BMW_컨버터블_6시리즈', 'BMW_쿠페_2시리즈', 'BMW_쿠페_4시리즈', 'BMW_쿠페_6시리즈', 'BMW_해치백_1시리즈', 'BMW_해치백_2시리즈투어러', '기아자동차_SUV_니로', '기아자동차_SUV_니로_EV', '기아자동차_SUV_모하비', '기아자동차_SUV_셀토스', '기아자동차_SUV_스토닉', '기아자동차_SUV_스포티지', '기아자동차_SUV_쏘렌토', '기아자동차_SUV_쏘울', '기아자동차_SUV_카렌스', '기아자동차_세단_K3', '기아자동차_세단_K5', '기아자동차_세단_K7', '기아자동차_세단_K9', '기아자동차_세단_로체', '기아자동차_세단_스팅어', '기아자동차_세단_스펙트라', '기아자동차_세단_쎄라토', '기아자동차_세단_오피러스', '기아자동차_세단_옵티마', '기아자동차_세단_포르테', '기아자동차_세단_프라이드', '기아자동차_승합_카니발', '기아자동차_쿠페_K3', '기아자동차_쿠페_포르테', '기아자동차_해치백_레이', '기아자동차_해치백_모닝', '기아자동차_해치백_프라이드', '닛산_SUV_로그', '닛산_SUV_엑스트레일', '닛산_SUV_쥬크', '닛산_SUV_큐브', '닛산_SUV_패스파인더', '닛산_세단_맥시마', '닛산_세단_알티마', '닛산_해치백_리프', '도요타_SUV_라브4', '도요타_세단_아발론', '도요타_세단_캠리', '도요타_해치백_프리우스', '도요타_해치백_프리우스C', '랜드로버_SUV_디스커버리', '랜드로버_SUV_레인지로버', '랜드로버_SUV_이보크', '렉서스_SUV_NX', '렉서스_SUV_RX', '렉서스_세단_ES', '렉서스_세단_LS', '렉서스_해치백_CT', '르노삼성_SUV_QM3', '르노삼성_SUV_QM5', '르노삼성_SUV_QM6', '르노삼성_SUV_XM3', '르노삼성_세단_SM3', '르노삼성_세단_SM5', '르노삼성_세단_SM6', '르노삼성_세단_SM7', '미니_SUV_컨트리맨', '미니_SUV_컨트리맨JWC', '미니_SUV_클럽맨', '미니_해치백_쿠퍼', '버스_버스_대형', '버스_버스_중,소형', '벤츠_SUV_GLA클래스', '벤츠_SUV_GLC클래스', '벤츠_SUV_GLE클래스', '벤츠_SUV_GLK클래스', '벤츠_SUV_GLS클래스', '벤츠_세단_A클래스', '벤츠_세단_CLA클래스', '벤츠_세단_CLS클래스', '벤츠_세단_C클래스', '벤츠_세단_E클래스', '벤츠_세단_S클래스', '벤츠_쿠페_C클래스', '벤츠_쿠페_E클래스', '볼보_SUV_XC40', '볼보_SUV_XC60', '볼보_SUV_XC90', '볼보_세단_S60', '볼보_세단_S90', '볼보_해치백_V40', '볼보_해치백_V60', '볼보_해치백_V90', '쉐보레_대우_SUV_대우_윈스톰', '쉐보레_대우_SUV_올란도', '쉐보레_대우_SUV_캡티바', '쉐보레_대우_SUV_트레일블레이저', '쉐보레_대우_SUV_트렉스', '쉐보레_대우_세단_대우_라세티', '쉐보레_대우_세단_대우_토스카', '쉐보레_대우_세단_대우_프린스', '쉐보레_대우_세단_말리부', '쉐보레_대우_세단_볼트하이브리드', '쉐보레_대우_세단_아베오', '쉐보레_대우_세단_임팔라', '쉐보레_대우_세단_지엠_알페온', '쉐보레_대우_세단_크루즈', '쉐보레_대우_해치백_마티즈', '쉐보레_대우_해치백_볼트EV', '쉐보레_대우_해치백_스파크', '쉐보레_대우_해치백_아베오', '쌍용자동차_SUV_렉스턴', '쌍용자동차_SUV_무쏘', '쌍용자동차_SUV_엑티언', '쌍용자동차_SUV_카이런', '쌍용자동차_SUV_코란도', '쌍용자동차_SUV_코란도 투리스모', '쌍용자동차_SUV_코란도스포츠', '쌍용자동차_SUV_티볼리', '쌍용자동차_세단_체어맨', '쌍용자동차_세단_체어맨H', '쌍용자동차_세단_체어맨W', '아우디_SUV_Q3', '아우디_SUV_Q5', '아우디_SUV_Q7', '아우디_SUV_Q8', '아우디_SUV_e트론', '아우디_세단_A4', '아우디_세단_A6', '아우디_세단_A7', '아우디_쿠페_A5', '이륜차_이륜차_대형', '이륜차_이륜차_중,소형', '인피니티_SUV_Q30', '인피니티_SUV_QX30', '인피니티_SUV_QX50', '인피니티_SUV_QX60', '인피니티_세단_Q70', '인피니티_쿠페_Q60', '재규어_SUV_E', '재규어_SUV_F', '재규어_세단_XE', '재규어_세단_XF', '제네시스_SUV_GV80', '제네시스_세단_EQ900', '제네시스_세단_G70', '제네시스_세단_G80', '제네시스_세단_G90', '제네시스_세단_제네시스', '제네시스_쿠페_제네시스 쿠페', '지프_SUV_그랜드체로키', '지프_SUV_랭글러', '지프_SUV_레니게이드', '지프_SUV_체로키', '지프_SUV_컴패스', '테슬라_SUV_모델X', '테슬라_세단_모델3', '테슬라_세단_모델S', '트럭_트럭_기타', '트럭_트럭_덤프트럭', '트럭_트럭_레미콘(믹서)', '트럭_트럭_카고트럭', '트럭_트럭_탱크로리', '트럭_트럭_트레일러', '특수_버스', '특수_특수_건설', '특수_특수_공공', '특수_특수_기타', '특수_화물_기아_봉고', '특수_화물_기타', '특수_화물_현대_포터', '포드_SUV_익스플로러', '포드_세단_몬데오', '포드_세단_토러스', '포드_컨버터블_머스탱', '포드_쿠페_머스탱', '포르쉐_SUV_카이엔', '포르쉐_세단_파나메라', '폭스바겐_SUV_투아렉', '폭스바겐_SUV_티구안', '폭스바겐_세단_CC', '폭스바겐_세단_아테온', '폭스바겐_세단_제타', '폭스바겐_세단_파사트', '폭스바겐_세단_페이톤', '폭스바겐_쿠페_비틀', '폭스바겐_해치백_골프', '푸조_SUV_2008', '푸조_SUV_3008', '푸조_SUV_5008', '푸조_세단_508', '푸조_해치백_308', '현대자동차_SUV_맥스크루즈', '현대자동차_SUV_베뉴', '현대자동차_SUV_베라크루즈', '현대자동차_SUV_싼타페', '현대자동차_SUV_코나', '현대자동차_SUV_테라칸', '현대자동차_SUV_투싼', '현대자동차_SUV_팰리세이드', '현대자동차_세단_그랜저', '현대자동차_세단_베르나', '현대자동차_세단_쏘나타', '현대자동차_세단_아반떼', '현대자동차_세단_아슬란', '현대자동차_세단_에쿠스', '현대자동차_승합_그랜드스타렉스', '현대자동차_해치백_i30', '현대자동차_해치백_i40', '현대자동차_해치백_벨로스터', '현대자동차_해치백_아이오닉', '현대자동차_해치백_엑센트', '혼다_SUV_CR', '혼다_SUV_파일럿', '혼다_세단_어코드', '혼다_승합_오딧세이']
# Transformations for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalization for RGB images
])

# Function to load the model
def load_model(model_path, device):
    model = efficientnet_b0(pretrained=False)
    # Load the state dict
    state_dict = torch.load(model_path, map_location=device)

    # Adjust the model's fc layer to match the saved model's fc layer
    num_classes = state_dict['classifier.1.weight'].shape[0]
    model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

    # Load the state dict into the model
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    return model

# Function to predict the class of an image
def predict_image(model, image_path, transform, device):
    image = Image.open(image_path).convert("RGB")  # Open the image and convert to RGB
    image = transform(image).unsqueeze(0)  # Apply the transformations and add batch dimension
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]

    return predicted_class

# Load the model
model = load_model(config["model_path"], config["device"])

# Path to the input image
image_path = "/content/SUV_볼보_XC40_004.jpg"  # Replace with your image path

# Predict the class of the input image
predicted_class = predict_image(model, image_path, transform, config["device"])
print(f"The predicted class is: {predicted_class}")
